# 导入工具

In [1]:
import warnings
warnings.filterwarnings('ignore')
import sys
import os
import json
from smart_open import smart_open

TEMP_FOLDER = os.path.abspath('./temp/')
print(f"文件夹 '{TEMP_FOLDER}' 将被用来存储语料和临时性字典")

import logging
logging.basicConfig(format="%(asctime)s: %(levelname)s: %(message)s", level=logging.INFO)

import numpy as np
np.set_printoptions(suppress=True)
import pandas as pd

import seaborn as sns
sns.set()
import matplotlib.pyplot as  plt
import matplotlib.patches as patches
plt.rcParams['font.sans-serif'] = ['SimHei']
%matplotlib inline

import tensorflow as tf
print(tf.__version__)
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, SpatialDropout1D, add, concatenate
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Bidirectional, LSTM
from tensorflow.keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D
from tensorflow.keras.preprocessing import text, sequence

from gensim.models import KeyedVectors
from gensim import models

import jieba

文件夹 'D:\Github\NLP\Artificial_Intelligence_for_NLP\Projects\3_commentClassification\temp' 将被用来存储语料和临时性字典
2.0.0


I1110 23:09:09.120841 15152 textcleaner.py:37] 'pattern' package not found; tag filters are not available for English


# Helper Functions

In [3]:
def fetch_stopwords(filename):
    stopwords = [line.strip() for line in smart_open(filename, 'r', encoding='utf-8')]
    return stopwords

In [97]:
stopwords = fetch_stopwords('data/stopwords.txt')
stopwords.append('\n')
print(stopwords[:10])
# 提高查询速度
stopwords = set(stopwords)

['!', '"', '#', '$', '%', '&', "'", '（', '）', '(']


In [101]:
def clean_sentence(content):
    content = jieba.cut(content)
    return [word for word in content if word not in stopwords if word.strip()]

In [126]:
def build_matrix(word_index, model):
    embedding_matrix = np.zeros((len(word_index)+1, model.wv.vectors.shape[1]))
    for word, index in word_index.items():
        try:
            embedding_matrix[index] = model.wv[word.lower()]
        except:
            pass
    return embedding_matrix

# 数据处理

## 词向量的选择
- 已有的
- 针对项目内容训练的（词频大于5的不存在OOV问题）

In [2]:
train_df = pd.read_csv('data/training.csv')
validation_df = pd.read_csv('data/validation.csv')
test_df = pd.read_csv('data/test.csv')

In [3]:
train_df.head()

,id,content,location_traffic_convenience,location_distance_from_business_district,location_easy_to_find,service_wait_time,service_waiters_attitude,service_parking_convenience,service_serving_speed,price_level,...,environment_decoration,environment_noise,environment_space,environment_cleaness,dish_portion,dish_taste,dish_look,dish_recommendation,others_overall_experience,others_willing_to_consume_again
0,0,"""吼吼吼，萌死人的棒棒糖，中了大众点评的霸王餐，太可爱了。一直就好奇这个棒棒糖是怎么个东西，...",-2,-2,-2,-2,1,-2,-2,-2,...,-2,-2,-2,-2,-2,-2,1,-2,1,-2
1,1,"""第三次参加大众点评网霸王餐的活动。这家店给人整体感觉一般。首先环境只能算中等，其次霸王餐提...",-2,-2,-2,-2,-2,-2,-2,0,...,0,0,0,0,1,-2,-2,-2,1,-2
2,2,"""4人同行 点了10个小吃\n榴莲酥 榴莲味道不足 松软 奶味浓\n虾饺 好吃 两颗大虾仁\...",-2,-2,-2,-2,0,-2,1,0,...,-2,-2,1,-2,0,1,-2,-2,0,-2
3,3,"""之前评价了莫名其妙被删 果断继续差评！ 换了菜单 价格更低 开始砸牌子 但套餐还是有150...",-2,-2,-2,-2,-2,-2,-2,0,...,-2,-2,-2,-2,-2,-1,-2,-2,-1,-1
4,4,"""出乎意料地惊艳，椰子鸡清热降火，美容养颜，大大满足了爱吃火锅怕上火星人。椰子冻是帅帅的老板...",-2,-2,-2,-2,-2,-2,-2,-2,...,-2,-2,-2,-2,-2,1,1,-2,1,-2


In [140]:
train_df.columns

Index(['id', 'content', 'location_traffic_convenience',
       'location_distance_from_business_district', 'location_easy_to_find',
       'service_wait_time', 'service_waiters_attitude',
       'service_parking_convenience', 'service_serving_speed', 'price_level',
       'price_cost_effective', 'price_discount', 'environment_decoration',
       'environment_noise', 'environment_space', 'environment_cleaness',
       'dish_portion', 'dish_taste', 'dish_look', 'dish_recommendation',
       'others_overall_experience', 'others_willing_to_consume_again'],
      dtype='object')

In [65]:
train_size, validation_size, test_size = len(train_df), len(validation_df), len(test_df)

In [32]:
full_content = pd.concat([train_df.content, validation_df.content, test_df.content],
                        axis=0, ignore_index=True).to_frame()
full_content.tail()

,content
134995,"""杭州大厦周围的日料店有三上、山葵这样的连锁店，听说三上除了C座地下一楼的那家以外还要在D座..."
134996,"""非常物美价廉的一家自助餐厅，虽是中午，之前担心的货少量不足的问题完全没出现，最爱的三文鱼、..."
134997,"""生日的时候不知道去吃什么，偶然在大牌抢购里发现了这家店～感觉团购很给力，就果断团了一份～地..."
134998,"""每次来大理都有不一样的感动，还记得上次来大理遇到一个特别好的出租车司机，这次来大理同样是这..."
134999,"""这家店位于城中湖码头，靠近广场非常近，离我们的住处比较近，开车过来只要几分钟的时间就好了。..."


In [103]:
%time full_content['clean_content'] = full_content.content.apply(clean_sentence)

Wall time: 7min 40s


In [104]:
full_content.to_csv('data/full_clean_content.csv', index=False)

In [109]:
%time model = models.Word2Vec(full_content.clean_content, size=300)

I1029 23:43:59.048060  4732 word2vec.py:1587] collecting all words and their counts
I1029 23:43:59.053062  4732 word2vec.py:1572] PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
I1029 23:43:59.325059  4732 word2vec.py:1572] PROGRESS: at sentence #10000, processed 1018024 words, keeping 65721 word types
I1029 23:43:59.589059  4732 word2vec.py:1572] PROGRESS: at sentence #20000, processed 2026955 words, keeping 94570 word types
I1029 23:43:59.857060  4732 word2vec.py:1572] PROGRESS: at sentence #30000, processed 3032507 words, keeping 116549 word types
I1029 23:44:00.138061  4732 word2vec.py:1572] PROGRESS: at sentence #40000, processed 4046062 words, keeping 135322 word types
I1029 23:44:00.424062  4732 word2vec.py:1572] PROGRESS: at sentence #50000, processed 5054530 words, keeping 152216 word types
I1029 23:44:00.697063  4732 word2vec.py:1572] PROGRESS: at sentence #60000, processed 6062940 words, keeping 167689 word types
I1029 23:44:00.980059  4732 word2vec.py:1572

Wall time: 2min 23s


In [110]:
model.save('data/full_content_wv.model')

I1029 23:47:41.675059  4732 utils.py:542] saving Word2Vec object under data/full_content_wv.model, separately None
I1029 23:47:41.677059  4732 utils.py:616] storing np array 'vectors' to data/full_content_wv.model.wv.vectors.npy
I1029 23:47:42.356058  4732 utils.py:648] not storing attribute vectors_norm
I1029 23:47:42.358058  4732 utils.py:616] storing np array 'syn1neg' to data/full_content_wv.model.trainables.syn1neg.npy
I1029 23:47:42.863063  4732 utils.py:648] not storing attribute cum_table
I1029 23:47:43.109059  4732 utils.py:556] saved data/full_content_wv.model


In [111]:
model.wv.vectors.shape

(68209, 300)

In [112]:
tokenizer = text.Tokenizer(num_words=len(model.wv.vocab), oov_token="<oov>")
tokenizer.fit_on_texts(full_content.clean_content)

In [113]:
word_index = tokenizer.word_index

In [114]:
full_sequences = tokenizer.texts_to_sequences(full_content.clean_content)

In [118]:
padded_sequences = sequence.pad_sequences(full_sequences, maxlen=500,
                                         padding='post', truncating='post')

In [127]:
embedding_matrix = build_matrix(word_index, model)

In [148]:
embedding_matrix.shape

(252717, 300)

In [141]:
train_data = padded_sequences[:train_size,:]
train_label = train_df.drop(['id', 'content'], axis=1).values

validation_data = padded_sequences[train_size:train_size+validation_size,:]
validation_label = validation_df.drop(['id', 'content'], axis=1).values

test_data = padded_sequences[-test_size:, :]

train_data.shape, validation_data.shape, test_data.shape, train_label.shape, validation_label.shape

((105000, 500), (15000, 500), (15000, 500), (105000, 20), (15000, 20))

# 模型

In [184]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(.1,
                                                         decay_steps=10000,
                                                         decay_rate=.96,
                                                         staircase=True)
csv_logger = tf.keras.callbacks.CSVLogger('temp/csv_logger')
stop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_acc',
                                                     mode='max',
                                                     patience=2)
checkpoint_dir = 'temp/training_checkpoints/'
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_dir,
                                                     monitor='val_acc',
                                                     verbose=1,
                                                     save_best_only=True)

In [76]:
# model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

In [186]:
def build_model():
    model = Sequential([
        Embedding(*embedding_matrix.shape, weights=[embedding_matrix], 
                  trainable=False, input_length=500),
        SpatialDropout1D(.2),
        Bidirectional(LSTM(128, return_sequences=True)),
        GlobalAveragePooling1D(),
        Dense(4*128, activation='relu'),
        Dense(4*64, activation='relu'),
        Dense(20, activation='softmax')
    ])
    
    model.compile(loss='categorical_crossentropy', 
                  optiizer=tf.keras.optimizers.SGD(lr_schedule), 
                  metrics=['accuracy'])
    return model

In [187]:
cl_model = build_model()
cl_model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 500, 300)          75815100  
_________________________________________________________________
spatial_dropout1d_10 (Spatia (None, 500, 300)          0         
_________________________________________________________________
bidirectional_7 (Bidirection (None, 500, 256)          439296    
_________________________________________________________________
global_average_pooling1d_10  (None, 256)               0         
_________________________________________________________________
dense_26 (Dense)             (None, 512)               131584    
_________________________________________________________________
dense_27 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_28 (Dense)             (None, 20)               

In [ ]:
history = cl_model.fit(train_data, train_label, batch_size=128, 
            epochs=20, validation_data=(validation_data, validation_label),
            callbacks=[csv_logger, stop_callback, cp_callback])